## Important notes:

To run the following code you'll need to activate the `abs_env` environment detailed in the `environment.yml` file in this repo.

Note: Not all cells here are intended to run within the notebook. You'll need to create new files as instructed, don't forget to also create an `__init__.py` file for properly importing the modules.

# 3. Design Patterns

Design patterns are solutions to commonly found problems in software design. They are tested paradigms for how to solve a recurring problem. Patterns allow to speed up software development and provide a uniform communication mechanism among developers.

There are various types of patterns:

    Creational: They deal with object or class creation.
                They help make a system independent of how its objects are created, composed and represented.

    Structural: They are about class or object composition.
                They define ways to compose objects to form larger structures and obtain new functionality.

    Behavioral: They are concerned with the assignment of responsibilities between objects
                and the communicaton between objects or classes.


## 3.1. The Adapter Design Pattern

The Adapter design pattern is a method of constructing a wrapper around a pre-existing interface to allow its use by another class. We can construct an individual adapter for a specific class, or create an abstract interface to base our adapter on. An abstract interface will allow us to swap out our adapter for any class that inherits from the interface.

An adapter is most useful in two different scenarios. The first is when you would like to re-use pre-existing code that has been rigorously tested without modifying it and thus invalidating the tests. The second is when you would like to create code that cares more about getting correct results for a set of inputs than how those results are generated. You may want to create a set of python libraries that can generate the results, but you do not want to re-write your main class to suit each new library.

As an example, consider the situation where we would like to read a Molecular Dynamics (MD) trajectory (i.e., concequtive xyz's) from a PDB file and perform some analysis on it. We currently have an MD library in mind to use, but we may want to use one or more different libraries in the future. If we do not want to re-write our code for each individual library, we need to come up with a different solution, in this case it will be to create an adapter.

We will use two MD libraries, [MDAnalysis](https://www.mdanalysis.org/) and [MDTraj](http://mdtraj.org/1.9.0/) in this example. Our goal is to create a code that uses `MDTraj` or `MDAnalysis` to perform some simple analysis on a trajectory file. To keep the runtime short, we will only use the center of mass and the [radius of gyration](https://en.wikipedia.org/wiki/Radius_of_gyration) from each code, but the concepts shown will apply to more in depth computations. A sample trajectory file to use is provided in PDB form. We will also be using `NumPy` to structure some of our data.

Let's first work with these libraries before turning them into adapters.

We'll start with `mdtraj`:

In [ ]:
import mdtraj as mdt

trajectory = mdt.load_pdb('protein.pdb')

center_of_mass = mdt.compute_center_of_mass(trajectory)
gyradius = mdt.compute_rg(trajectory)

print(f'Center of mass:\n{center_of_mass}')
print(f'Radius of Gyration:\n{gyradius}')


Let's assume that we now want to do the same using a different library, maybe because it ha a better reputation, or that we no longer have a license for the first one, or maybe the second has additional important features.

Let us quickly change the code to use `MDAnalysis` to load the PDB file and perform our two computations:

In [ ]:
import MDAnalysis as mda
import numpy as np

trajectory = mda.Universe('protein.pdb')

mass_by_frame = np.ndarray(shape=(len(trajectory.trajectory), 3))
for ts in trajectory.trajectory:
    mass_by_frame[ts.frame] = trajectory.atoms.center_of_mass(compound='segments')

rg_by_frame = np.empty(len(trajectory.trajectory))
for ts in trajectory.trajectory:
    rg_by_frame[ts.frame] = trajectory.atoms.radius_of_gyration()

print(f'Center of mass:\n{mass_by_frame}')
print(f'Radius of Gyration:\n{rg_by_frame}')


Note that the implementation detailes (computing `center_of_mass` and `gyradius`) have changed.

the two libraries handle the operations very differently. The function definitions are completely distinct from one another.

We can see that not only are the function definitions different, both the output structure and the units are different too... `MDTraj` is outputting in nanometers and `MDAnalysis` is using Ångströms (we'll fix that later). Also, `MDTraj` outputs all frames, while we had to manually ask `MDAnalysis` to run the calculation for each frame seperately in a loof, then append the result.

## 3.2. Trajectory Adapter

We want to code towards an interface, not a specific class, so we want to create an interface to base our adapters on.
We will use Python's abc module to help build the interface, so we will need to import it and build our interface.

Let's write the following implementation in a new file called `adapter.py` to hold our adapter.

In [ ]:
from abc import abstractmethod, ABC

class TrajectoryAdapter(ABC):

    @abstractmethod
    def compute_center_of_mass():
        pass

    @abstractmethod
    def compute_radius_of_gyration():
        pass

Inheriting from ABC and decorating the methods with `@abstractmethod` ensures that any subclass of TrajectoryAdapter must override both methods. Any code developed using the listed abstract methods from the interface will now work with any adapter we construct that inherits from `TrajectoryAdapter`.

### 3.2.1. MDTraj Adapter

We will start by building an Adapter that utilizes MDTraj. Let's create a new file called `mdtra_adapter.py`.

In [ ]:
from adapter import TrajectoryAdapter
import mdtraj as mdt

class MDTrajAdapter(TrajectoryAdapter):
    def __init__(self, filename):
        self.trajectory = md.load_pdb(filename)
        print('Selected MDTraj.')

    def compute_center_of_mass(self):
        return 10 * md.compute_center_of_mass(self.trajectory)

    def compute_radius_of_gyration(self):
        return 10 * md.compute_rg(self.trajectory)

### 3.2.2. MDAnalysis Adapter

Now we would like to change our code to use the MDAnalysis library, ideally with minimal impact on our script, which is simulating a larger code. Let us construct another Adapter for MDAnalysis. Create a new file called `mdanalysis_adapter.py` with the following content:

In [ ]:
from adapter import TrajectoryAdapter
import MDAnalysis as mda
import numpy as np

class MDAnalysisAdapter(TrajectoryAdapter):
    def __init__(self, filename):
        self.trajectory = MDAnalysis.Universe(filename)
        print('Selected MDAnalysis.')

    def compute_center_of_mass(self):
        mass_by_frame = np.ndarray(shape=(len(self.trajectory.trajectory), 3))
        for ts in self.trajectory.trajectory:
            mass_by_frame[ts.frame] = self.trajectory.atoms.center_of_mass(compound='segments')
        return mass_by_frame

    def compute_radius_of_gyration(self):
        rg_by_frame = np.empty(len(self.trajectory.trajectory))
        for ts in self.trajectory.trajectory:
            rg_by_frame[ts.frame] = self.trajectory.atoms.radius_of_gyration()
        return rg_by_frame

It is the programmer's responsibility to make sure all overidden methods return the same type of variables (having the same units)

### 3.2.3. Run the adapters

Run the following pieces of code (make sure to import correctly, i.e., by placing all python files in the same folder):

In [ ]:
from mdtraj_adapter import MDTrajAdapter

mda = MDTrajAdapter('protein.pdb')
print(f'Center of mass:\n{mda.compute_center_of_mass()}')
print(f'Radius of Gyration:\n{mda.compute_radius_of_gyration()}')

In [ ]:
from mdanalysys_adapter import MDAnalysisAdapter

mda = MDAnalysisAdapter('protein.pdb')
print(f'Center of mass:\n{mda.compute_center_of_mass()}')
print(f'Radius of Gyration:\n{mda.compute_radius_of_gyration()}')

Note how elegant the above code is. We have abtracted out all implementation details, we just need to call the coirrect adapter, and now the input, the called methods, and the output structure are all identical!

With adapters for each library, our code is not concerned with how the data it needs is generated, simply that it follows the contract put in place by the interface.

## 3.3. Factory Design Pattern

While the above solution was indeed starting to look elegant, we still had to provide the specific adapter ourselves... We solve this by defining a Factory.

The Factory design pattern is a method that uses a superclass to provide a common interface for the creation of related objects. Subclasses decide which object type should be created and of hide the implementation details of such objects.

### The problem:

Consider the adapters we previously designed for the two different trajectory analysis tools: `MDAnalysis` and `MDTraj`. Currently we only have two different adapters built, but who knows how many may be constructed in the future. Modifying code to select just between a few toolkits already starts to get tedious if you have to change it a lot. It would be much easier to just take the name of the toolkit you want to use _as a parameter_ and not have to change any code. We want to create a single interface that remains constant that encapsulate the creation of the adapters and abstracts the specifics away.

### The solution:

The Factory design pattern suggests to replace direct object construction calls (i.e., instantiating the `MDAnalysisAdapter` or `MDTrajAdapter` classes) with calls to a special class called a factory. This class has a method which returns instances of each adapter and hides the specific instantiation of each. A factory works best when the different constructed classes work from a shared base class, this allows code to use any object produced by the factory in the same way, without even knowing the type of the object they posses.


First we will want to create a new module for our factory, called `factory.py`. We need to first import our adapter abstract class so our factory knows which type of objects it is building.

In [ ]:
from adapter import TrajectoryAdapter
from mdanalysis_adapter import MDAnalysisAdapter
from mdtraj_adapter import MDTrajAdapter


_registered_trajectory_adapters = {}


def register_trajectory_adapter(trajectory_adapter_label: str,
                                trajectory_adapter_class: Type[TrajectoryAdapter],
                               ) -> None:
    """
    A register for trajectory adapters.

    Args:
        trajectory_adapter_label (str): A string representation for a trajectory adapter.
        trajectory_adapter_class (TrajectoryAdapter): The trajectory adapter class (a child of TrajectoryAdapter).

    Raises:
        TypeError: If trajectory_adapter_class is not a subclass of TrajectoryAdapter.
    """
    if not issubclass(trajectory_adapter_class, TrajectoryAdapter):
        raise TypeError(f'Adapter class {_registered_trajectory_adapters} is not a subclass TrajectoryAdapter.')
    _registered_trajectory_adapters[trajectory_adapter_label] = trajectory_adapter_class


def trajectory_factory(trajectory_adapter: str,
                       filename: str,
                      ) -> TrajectoryAdapter:
    """
    A factory generating a trajectory adapter corresponding to ``trajectory_adapter``.

    Args:
        trajectory_adapter (str): The string representation of the trajectory adapter.
        filename (str): The trajectory file name to analyze (will be passed to the respective adapter).

    Returns:
        TrajectoryAdapter: The requested TrajectoryAdapter subclass, initialized with the respective arguments.
    """
    if trajectory_adapter not in _registered_trajectory_adapters.keys():
        raise ValueError(f'The trajectory_adapter argument of {trajectory_adapter} was not present in the keys for the '
                         f'_registered_trajectory_adapters dictionary: {list(_registered_trajectory_adapters.keys())}'
                         f'\nPlease check that the trajectory adapter was registered properly.')

    traj_adapter_class = _registered_trajectory_adapters[trajectory_adapter](filename=filename)
    return traj_adapter_class

Now all we need to do is make sure all our adapters are properly "registered" in the `_registered_trajectory_adapters` dictionary. We need to add an `__init__.py` file for this small repository, so all the files are loaded properly:

In [ ]:
import adapter
import factory
import mdtra_adapter
import mdanalysis_adapter

And now let's add the following line in each of the adapters (this is an example for `MDTrajAdapter`, generate a respective line for  `MDAnalysisAdapter`). Place these lines at the end of each adapter file. Since they are not inside a class or a function they will get immediately executed when our small program initializesL

In [ ]:
# Add this line at the top:
from factory import register_trajectory_adapter

# Add this line at the bottom of the adapter file (modufy as needed):
register_trajectory_adapter('mdtraj', MDTrajAdapter)

We should be all set by now. Let's now elegantly call factory, pass it a string that represents our adapter, and see how it works:

In [ ]:
from factory import trajectory_factory

traj_adapter = trajectory_factory(trajectory_adapter='mdtraj', filename='protein.pdb')

print(f'Center of mass:\n{traj_adapter.compute_center_of_mass()}')
print(f'Radius of Gyration:\n{traj_adapter.compute_radius_of_gyration()}')

In [ ]:
from factory import trajectory_factory

traj_adapter = trajectory_factory(trajectory_adapter='mdanalysis', filename='protein.pdb')  # Replace "mdanalysis" with the name you gave to the second adapter, if different

print(f'Center of mass:\n{traj_adapter.compute_center_of_mass()}')
print(f'Radius of Gyration:\n{traj_adapter.compute_radius_of_gyration()}')